In [72]:
import pandas as pd 
import numpy as np 
from sklearn.model_selection import train_test_split
from collections import defaultdict

In [1]:
df = pd.read_csv('./IMDb movies.csv')
df.head()

/home/davis/Documents/venv/hw/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3437: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,imdb_title_id,title,original_title,year,date_published,genre,duration,country,language,director,...,actors,description,avg_vote,votes,budget,usa_gross_income,worlwide_gross_income,metascore,reviews_from_users,reviews_from_critics
0,tt0000009,Miss Jerry,Miss Jerry,1894,1894-10-09,Romance,45,USA,None,Alexander Black,...,"Blanche Bayliss, William Courtenay, Chauncey D...",The adventures of a female reporter in the 1890s.,5.9,154,NaN,NaN,NaN,NaN,1.0,2.0
1,tt0000574,The Story of the Kelly Gang,The Story of the Kelly Gang,1906,1906-12-26,"Biography, Crime, Drama",70,Australia,None,Charles Tait,...,"Elizabeth Tait, John Tait, Norman Campbell, Be...",True story of notorious Australian outlaw Ned ...,6.1,589,$ 2250,NaN,NaN,NaN,7.0,7.0
2,tt0001892,Den sorte drøm,Den sorte drøm,1911,1911-08-19,Drama,53,"Germany, Denmark",NaN,Urban Gad,...,"Asta Nielsen, Valdemar Psilander, Gunnar Helse...",Two men of high rank are both wooing the beaut...,5.8,188,NaN,NaN,NaN,NaN,5.0,2.0
3,tt0002101,Cleopatra,Cleopatra,1912,1912-11-13,"Drama, History",100,USA,English,Charles L. Gaskill,...,"Helen Gardner, Pearl Sindelar, Miss Fielding, ...",The fabled queen of Egypt's affair with Roman ...,5.2,446,$ 45000,NaN,NaN,NaN,25.0,3.0
4,tt0002130,L'Inferno,L'Inferno,1911,1911-03-06,"Adventure, Drama, Fantasy",68,Italy,Italian,"Francesco Bertolini, Adolfo Padovan",...,"Salvatore Papa, Arturo Pirovano, Giuseppe de L...",Loosely adapted from Dante's Divine Comedy and...,7.0,2237,NaN,NaN,NaN,NaN,31.0,14.0


In [6]:
data = pd.read_csv('./filtered_data.csv')
data.head()

,movieId,userId,rating,description,genre,director,actors,country,original_title
0,30,900816,3,A swinger on the cusp of being a senior citize...,"Comedy, Drama, Romance",Nancy Meyers,"Jack Nicholson, Diane Keaton, Keanu Reeves, Fr...",USA,Something's Gotta Give
1,30,1990901,4,A swinger on the cusp of being a senior citize...,"Comedy, Drama, Romance",Nancy Meyers,"Jack Nicholson, Diane Keaton, Keanu Reeves, Fr...",USA,Something's Gotta Give
2,30,1402412,4,A swinger on the cusp of being a senior citize...,"Comedy, Drama, Romance",Nancy Meyers,"Jack Nicholson, Diane Keaton, Keanu Reeves, Fr...",USA,Something's Gotta Give
3,30,1601783,3,A swinger on the cusp of being a senior citize...,"Comedy, Drama, Romance",Nancy Meyers,"Jack Nicholson, Diane Keaton, Keanu Reeves, Fr...",USA,Something's Gotta Give
4,30,306466,3,A swinger on the cusp of being a senior citize...,"Comedy, Drama, Romance",Nancy Meyers,"Jack Nicholson, Diane Keaton, Keanu Reeves, Fr...",USA,Something's Gotta Give


In [49]:
imdb_df = data[['movieId','genre', 'description', 'original_title']]
imdb_df = imdb_df.drop_duplicates()

In [50]:
imdb_df = imdb_df.reset_index(drop=True)
imdb_df.head()

,movieId,genre,description,original_title
0,30,"Comedy, Drama, Romance",A swinger on the cusp of being a senior citize...,Something's Gotta Give
1,77,"Action, Adventure, Mystery",When an expedition to the African Congo ends i...,Congo
2,83,"Biography, Drama, History",A worker at a plutonium processing plant is pu...,Silkwood
3,108,"Action, Crime, Drama",The investigation into a kidnapping of the dau...,Spartan
4,118,"Action, Adventure, Thriller",Rambo returns to the jungles of Vietnam on a m...,Rambo: First Blood Part II


In [51]:
imdb_df.shape

(1478, 4)

In [52]:
imdb_df.dtypes

movieId            int64
genre             object
description       object
original_title    object
dtype: object

In [53]:
imdb_df['description'] = imdb_df['description'].fillna('')

In [54]:
imdb_df.isna().any()

movieId           False
genre             False
description       False
original_title    False
dtype: bool

In [55]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfv = TfidfVectorizer(min_df=3, max_features=None, strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}', ngram_range=(1,3), stop_words='english')

In [56]:
tfv_matrix = tfv.fit_transform(imdb_df['description'])

In [57]:
tfv_matrix

<1478x1965 sparse matrix of type '<class 'numpy.float64'>'
	with 15016 stored elements in Compressed Sparse Row format>

In [58]:
from sklearn.metrics.pairwise import sigmoid_kernel

sig = sigmoid_kernel(tfv_matrix, tfv_matrix)

In [59]:
sig[0]

array([0.7618078 , 0.76159416, 0.76159416, ..., 0.76159416, 0.76159416,
       0.76159416])

In [60]:
index_movie = dict(zip(list(range(imdb_df.shape[0])),list(imdb_df['movieId'].tolist())))
movie_index = dict(zip(list(imdb_df['movieId'].tolist()),list(range(imdb_df.shape[0]))))

In [61]:
index_movie[2]

83

In [62]:
sig[2]

array([0.76159416, 0.76159416, 0.7618078 , ..., 0.76159416, 0.76159416,
       0.76159416])

In [84]:
def recommend(movie, sig=sig):
    idx = movie_index[movie]

    sig_scores = list(enumerate(sig[idx]))

    sig_scores = sorted(sig_scores, key=lambda x: x[1], reverse=True)

    sig_scores = sig_scores[1:11]

    movie_indices = [i[0] for i in sig_scores]
    
    return set(imdb_df['movieId'].iloc[movie_indices])


In [44]:
import random 

random_movie = random.choice(list(movie_index.keys()))
random_movie

11152

In [64]:
recommend(random_movie)

1185    14459
1135    13840
1064    13052
901     11051
1306    15733
1282    15445
632      7544
2          83
789      9528
595      7048
Name: movieId, dtype: int64

In [65]:
good_movies = data[data['rating']>=4]

In [66]:
movie_titles = dict(zip(imdb_df['movieId'].tolist(),imdb_df['original_title'].tolist()))

In [73]:
train, test = train_test_split(good_movies, test_size=0.8, random_state=42, shuffle=True)
train = train.reset_index(drop=True)
test = test.reset_index(drop=True)

In [74]:
train.shape

(2783483, 9)

In [75]:
moviesPerUser = defaultdict(set)
usersPerMovie = defaultdict(set)
for i in range(train.shape[0]):
    movie = train['movieId'][i]
    user = train['userId'][i]
    moviesPerUser[user].add(movie)
    usersPerMovie[movie].add(user)

In [92]:
random_user = random.choice(list(moviesPerUser.keys()))
random_user

2382165

In [113]:
precision_sum = 0
for u in moviesPerUser:
    overall_recommendation = set()
    user_df = good_movies[good_movies['userId'] == u]
    liked_movies = set(user_df['movieId'])
    for m in moviesPerUser[random_user]:
        overall_recommendation = overall_recommendation.union(recommend(m))
    precision_sum += len(overall_recommendation & liked_movies)/len(liked_movies)

In [114]:
n_user = len(moviesPerUser)

In [115]:
watched_rate = precision_sum/n_user
watched_rate

0.26837086916475233

In [87]:
recommend(random_movie)

{299, 5946, 6960, 7364, 8339, 11218, 12546, 13865, 16082, 16793}